# SENNA для задачи POS-tagging

SENNA (Semantic/syntactic Extraction using a Neural Network Architecture) – архитектура нейронной сети, позволяющая достигнуть state-of-the-art результатов в нескольких задачах обработки текстов. 


#### Задача POS-tagging
ставится как задача многоклассовой классификации: 

* $T$ - количество различных тегов частей речи (каждое слово $w$ относится к одному из $T$ классов)
* для каждого слова из train формируется вектор признаков 
* NN обучается по всем векторам признаков каждого слова из train 

#### Подход к решению 
представлен в https://arxiv.org/pdf/1103.0398.pdf( Window approach network, раздел 3.3.1):
1. Каждое слово представляется эмбеддингом размерности $d$;
2. Для каждого слова формируется окно длины $k$ из $(k-1)/2$ соседних слов слева от данного слова  и $(k-1)/2$ соседних слов справа от данного слова, $k$ – нечетное. (Если для слова невозможно найти соседние слова, используется padding.)
3. Для каждого слова формируется вектор признаков, состоящий из конкатенированных эмбеддингов слов из левого окна, данного слова и слов из правого окна. Итоговая размерность вектора признаков – $d \times k$. Этот вектор подается на вход нейронной сети;
4. Обучается нейронная сеть, имеющая один скрытый слой с $n_h$ нейроннами и нелинейной функцией активации $\theta$;
5. На выходном слое нейронной сети решается задача классификации на |T| классов. 


Открытый корпус: https://github.com/dialogue-evaluation/morphoRuEval-2017/blob/master/OpenCorpora_Texts.rar

Предобученные эмбеддинги Facebook: https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki.ru.vec

In [ ]:
! wget https://www.dropbox.com/s/n5pgf9nu50jvwra/unamb_sent_14_6.conllu
! wget https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki.ru.vec

In [ ]:
! pip install nltk
! pip install tqdm
! pip install keras

### 1. Составляем обучающую выборку 

- считываем выборку 
- делим на train, test по предложениям
- каждое предложение внутри каждого из множества разделям на слова (оставляем структуру предложения в виде list, потому что нам потребуется контекст: слова слева и справа)

In [ ]:
path = 'unamb_sent_14_6.conllu'
project_path = '/content'

In [ ]:
from nltk.corpus.reader import ConllCorpusReader
pos_corpus = ConllCorpusReader(project_path, fileids = path, 
                               columntypes = ['ignore', 'words', 'ignore', 'pos', 'chunk'])
sents = list(pos_corpus.iob_sents())

In [ ]:
pos_tags = set([pos for text in sents for word, pos, chunk in text])

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(sents, test_size=0.25)
sent_train = # your code is here
label_train = # your code is here
sent_test = # your code is here
label_test = # your code is here

### 2. Считываем эмбеддинги

Далее будем подавать эмбеддинги на вход Embedding layer в поле weights (матрица). Матрица train при этом должна быть integer-encoded (слову соответствует индекс), т.е. строка матрицы - контекст слова, для которого есть POS-метка.

- будем хранить не все эмбеддинги, а только для слов, которые встречаются в train и test: надо сохранить сами эмбеддинги в матрицу (word_embeddings), и запомнить соответствие слово <-> индекс в матрице (word_2_idx)
- будьте внимательны с кодировкой .de(en)code('utf-8')
- не забудем о PADDING, UNKNOWN (для некоторых слов не существует контекста, в этом случае эмбеддинг будет из нулей; для некоторых слов не найдется предобученного эмбеддинга, создадим для таких слов эмбеддинг np.random.uniform)

In [ ]:
# чтобы знать, какие слова есть
words = set()

for sent_set in [sent_train, sent_test]:
    for sentence in sent_set:
        for token in sentence:
            words.add(token.lower())

In [ ]:
from tqdm import tqdm
import numpy as np

word_2_idx = {}
word_embeddings = []

with open('wiki.ru.vec', 'rb') as f :
    for line in tqdm(f):
        values = line.split()
        if len(values) != 301:
            continue
            
        if len(word_2_idx) == 0:
            
            # your code is here
            word_2_idx["padding"] = 
            word_embeddings.append(vector)
            
            # your code is here
            word_2_idx["unknown"] = len(word_2_idx)
            word_embeddings.append(vector)

        if  values[0].lower().decode('utf-8') in words:
            # your code is here
            word_2_idx[values[0].lower()] = 
            word_embeddings.append(vector)
            
            
            
word_embeddings = np.array(word_embeddings)

In [ ]:
word_embeddings.shape, len(word_2_idx), len(words)

### 3. Составляем train

- сформируем окно для каждого слова размера $k$;
- закодируем каждое слово из контекста индексом, соответсвующим этому слову в матрице эмбеддингов
- не забываем про padding, unknown

### 4. Составляем test
- кодируем каждый label индексом

In [ ]:
WINDOWSIZE = 5
UNKNOWN_IDX = word_2_idx['unknown']
PADDING_IDX = word_2_idx['padding']

In [ ]:
def get_context(tgt_word_idx, sentence, windowsize):
    context = []  
    for word_position in range(tgt_word_idx - windowsize, tgt_word_idx + windowsize+1):
        if word_position < 0 or word_position >= len(sentence):
            # your code is here
            
        context.append(word)   
    return context


# сюда будем записывать не сами слова, а индекс эмбеддинга
X_train = []

for sentence in sent_train:
    for tgt_word_idx in range(len(sentence)):
        tgt_word_context = get_context(tgt_word_idx, sentence, WINDOWSIZE/2)
        # your code is here
        # добавляем все индексы слов из контекста, не забываем заполнять unknown ords
        X_train.append()

        
label_2_idx = {}
for label in pos_tags:
    label_2_idx[label] = len(label_2_idx)
y_train = []
for el in label_train:
    # your code is here
    y_train.extend()
    
X_train = np.array(X_train)
y_train = np.array(y_train)

### 5. Обучаем NN


In [ ]:
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Dropout, Activation, Flatten
from keras.layers import Embedding

n_in = X_train.shape[1] # windowsize
n_out = len(label_2_idx) # num of labels


# your code is here
# trainable=False using fixed embeddings for a text input
words_input = Input(shape=(n_in,), dtype='int32', name='words_input')
words = Embedding()(words_input)
words = Flatten()(words)

output = Dense(64, activation=)(words)
output = Dense(n_out, activation=)(output)

model = Model(input=[words_input], output=[output])
model.compile(loss='sparse_categorical_crossentropy', optimizer='nadam')
model.summary()

In [ ]:
model.fit(X_train, y_train, nb_epoch= 2, batch_size = 32,  validation_split = 0.1)

### 6. Делаем предсказание для test
- составляем test
- применяем модель
- смотрим на качество

In [ ]:
X_test = []
# your code is here
       
y_test = []
# your code is here

X_test = np.array(X_test)
y_test = np.array(y_test)

In [ ]:
idx_2_label = {v: k for k, v in label_2_idx.items()}
pred = model.predict(X_test)
t = np.array([idx_2_label[i] for i in y_test])
p = np.array([idx_2_label[i] for i in np.argmax(pred, axis=1)])

In [ ]:
from sklearn.metrics import classification_report
print np.mean(t==p)
print classification_report(t, p)